In [1]:
import urllib.request as r
import bs4
import re
url = 'http://hip-hop.name/text/guf/'

# WARNING: Я не смог быть адекватным в этом задании и решил выкачать треки русского рэпера

In [2]:
response = r.urlopen(url)
html_doc = response.read()
soup = bs4.BeautifulSoup(html_doc, 'html.parser')

In [3]:
links = []
for link in soup.find_all('a'):
    links.append(link.get('href'))

In [4]:
links = [*filter(lambda x: x is not None and x[:4] == '/guf', links)]

In [5]:
texts_urls = ['http://hip-hop.name' + link for link in links]

In [6]:
%%time
learning_text = ''
for url in texts_urls:
    try:
        response = r.urlopen(url)
        html_doc = response.read()
        soup = bs4.BeautifulSoup(html_doc, 'html.parser')
        s = soup.find_all('div', 'entry')[0]
        learning_text += s.get_text()
    except:
        pass

CPU times: user 4.14 s, sys: 141 ms, total: 4.28 s
Wall time: 40.5 s


In [7]:
#!/usr/bin/env python3
import string
from collections import Counter, OrderedDict
import random
import argparse
import sys
import unittest


class Utils(object):

    @staticmethod
    def split_symbol(text, symbol, drop_symbol=False):
        if drop_symbol:
            return text.split(symbol)

        parts = text.split(symbol)
        result = []
        for part in parts:
            result += [part, symbol]
        return result[:-1]

    @staticmethod
    def split_for_tokens(text):
        splited = []
        number = ''
        word = ''
        other = ''
        for i in range(len(text)):
            if text[i].isdigit():
                if len(other) > 0:
                    splited += [other]
                    other = ''
                if len(word) > 0:
                    splited += [word]
                    word = ''
                number += text[i]
            elif text[i].isalpha():
                if len(other) > 0:
                    splited += [other]
                    other = ''
                if len(number) > 0:
                    splited += [number]
                    number = ''
                word += text[i]
            else:
                if len(word) > 0:
                    splited += [word]
                    word = ''
                if len(number) > 0:
                    splited += [number]
                    number = ''
                other += text[i]
        if len(word) > 0:
            splited += [word]
        if len(number) > 0:
            splited += [number]
        if len(other) > 0:
            splited += [other]
        return splited

    @staticmethod
    def parts_split_symbol(parts, symbol, drop_symbol=False):
        new_parts = []
        for part in parts:
            new_parts += Utils.split_symbol(part, symbol, drop_symbol)
        return [s for s in filter(lambda x: len(x) != 0, new_parts)]

    @staticmethod
    def split(text, delimiters, delimiters_to_drop=[]):
        """

        :param text: type(str)
        :param delimiters: to split text
        :param delimiters_to_drop: to split text and not include to result
        :return: splitted text with delimiters and w/o delimiters_to_drop
        """
        parts = [text]

        for symbol in delimiters_to_drop:
            parts = Utils.parts_split_symbol(parts, symbol, drop_symbol=True)

        for symbol in delimiters:
            parts = Utils.parts_split_symbol(parts, symbol, drop_symbol=False)

        new_parts = []
        for part in parts:
            new_parts += Utils.split_for_tokens(part)
        return new_parts

    @staticmethod
    def tokenize(text, depth=1, drop_whitespace=False):
        """

        :param text: type(str)
        :param depth: length of words chain
        :param drop_whitespace: if true, drops whitespaces in final list
        :return: list of tokens
        """
        delimiters = []
        delimiters_to_drop = []
        if drop_whitespace:
            delimiters = []
            delimiters_to_drop = string.whitespace + string.punctuation
        else:
            delimiters = string.punctuation + string.whitespace

        splited = Utils.split(text, delimiters, delimiters_to_drop)
        return [tuple(splited[i:i + depth]) for i in range(0, len(splited) - depth + 1)]

    @staticmethod
    def get_chains(text, depth=1):
        splited_text = text.split('\n')
        chains = dict()
        for line in splited_text:
            chains_with_future = Utils.tokenize(line, depth + 1, drop_whitespace=True)
            for item in chains_with_future:
                if item[:depth] not in chains:
                    chains[item[:depth]] = []
                chains[item[:depth]] += item[depth:]
        return chains

    @staticmethod
    def count_probabilities(chains):
        chains_with_probabilities = OrderedDict()
        for key, values in chains.items():
            counts = Counter(values)
            total = len(values)
            probabilities = OrderedDict()
            for v in values:
                probabilities[v] = counts[v] / total
            chains_with_probabilities[key] = OrderedDict(sorted(list(probabilities.items()),
                                                                key=lambda x: x[0])
                                                         )
        return OrderedDict(sorted(chains_with_probabilities.items(), key=lambda x: x[0]))

    @staticmethod
    def unique(l):
        """
        :param l:
        :return: unique elements of list
        """
        return list(set(l))


class TokenizeTask(object):
    def __init__(self, text):
        self.text = text
        self.tokens = Utils.tokenize(self.text)

    def __str__(self):
        return '\n'.join([s[0] for s in self.tokens])


class CalculateProbabilitiesTask(object):
    def __init__(self, text, depth=1):
        self.chains = [Utils.get_chains(text, depth=i) for i in range(0, depth + 1)]
        self.probabilities = [Utils.count_probabilities(chain) for chain in self.chains]

    def __str__(self):
        pr = []
        for d in self.probabilities:
            for key, values in d.items():
                pr += [(key, values)]
        pr = sorted(pr, key=lambda x: x[0])
        ans = []
        for pr_per_level in pr:
            history, values = pr_per_level[0], pr_per_level[1]
            ans.append(' '.join(history))
            for next_word, p in values.items():
                ans.append('  {}: {:.2f}'.format(next_word, p))
        return '\n'.join(ans)


class TextGenerator(object):
    def __init__(self, depth=1, size=20):
        self.depth = depth
        self.size = size
        self.probabilities = []

    def fit(self, text):
        prob_task = CalculateProbabilitiesTask(text, self.depth)
        self.probabilities = prob_task.probabilities

    def generate(self):
        generated_words = []
        new_word = self.__choice(self.probabilities[0][()])
        generated_words.append(new_word)

        for i in range(1, self.size):
            slice_size = min(self.depth - 1, i)
            prev_chain = tuple(generated_words[-slice_size:])
            while prev_chain not in self.probabilities[slice_size]:
                slice_size -= 1
                prev_chain = tuple(generated_words[-slice_size:])
                if (slice_size < 0):
                    slice_size = 0
                    prev_chain = ()
                    break
            new_word = self.__choice(self.probabilities[slice_size][prev_chain])
            generated_words.append(new_word)
        generated_words[0] = generated_words[0].capitalize()
        return ' '.join(generated_words)

    def __choice(self, future):
        total_prob = 0
        for next_word, p in future.items():
            total_prob += p
        result = random.uniform(0, total_prob)
        lower_bound = 0
        for next_word, p in future.items():
            lower_bound += p
            if lower_bound > result:
                return next_word


class UnitTest(unittest.TestCase):
    @staticmethod
    def test_tokenize():
        res = TokenizeTask('1234 5678, 9101112! We love BMW and you love')
        answer = [
                    '1234',
                    ' ',
                    '5678',
                    ',',
                    ' ',
                    '9101112',
                    '!',
                    ' ',
                    'We',
                    ' ',
                    'love',
                    ' ',
                    'BMW',
                    ' ',
                    'and',
                    ' ',
                    'you',
                    ' ',
                    'love'
                ]
        assert([x[0] for x in res.tokens] == answer)

    @staticmethod
    def test_probabilities_calculation():
        result = CalculateProbabilitiesTask('First test Second test', depth=1).__str__().split()
        answer = ['First:', '0.25',
                  'Second:', '0.25',
                  'test:', '0.50',
                  'First', 'test:', '1.00',
                  'Second', 'test:', '1.00',
                  'test', 'Second:', '1.00']
        assert(result == answer)

    @staticmethod
    def test_generation():
        try:
            generator = TextGenerator(depth=3, size=5)
            generator.fit("а блоки, каждый блок соответствует цепочке истории")
            generator.generate()
        except _:
            assert False

In [8]:
tg = TextGenerator(5, 200)

In [9]:
len(learning_text)

239309

In [10]:
tg.fit(learning_text)

Самое веселое (жестокое) (...Какого черта вообще мне это пришло в голову?)

In [11]:
rap_gad = tg.generate()
l = rap_gad.split(' ')
for i in range(0, len(l)):
    print(l[i], end=' ')
    if i % 6 == 0:
        print()

Мало 
того что не тонет ещё и 
воняет ровно бро я не готов 
давай потом не увижу ни разу 
не давали мне повода Лучше не 
трогай Гуфаку если он не пафал 
не светит нихуя позитивнго Фантазировать на 
тему всякой фигни вечер 25 октября 
2007 года я такой то такой 
то Да ладно проехали ведь мы 
тут а не там че трясет 
то так а Не могу остановиться 
чердаке рэп групп писал ей открыть 
рот шоубиз  — держись шоубиз  — 
держись своё слово заново понять лишение 
амбиций серьезно хочешь возьми ящик калашей 
иль Ноггано Гуф и 5 Плюх 
Москва так надо а О да 
я не могу представить сколько теперь 
будет вони в неё что нибудь 
брошу банку фанты из окна все 
та же высота четвертому вами братцы 
летим неумолимо с большого обрыва опух 
понедельник этого андерграунд представления утро всегда 
ахуеваю с похмелья мои два с 
половиной фаната на музыкальных каналах снег 
пушистый свеженький текст Любить себя меньше 
вальса больше гопака суёт в ебальник 
что хорошо ментов стало больше научились 
думать о чём 

In [12]:
rap_gad = tg.generate()
l = rap_gad.split(' ')
for i in range(0, len(l)):
    print(l[i], end=' ')
    if i % 6 == 0:
        print()

Искренне 
до талого то награждали что то 
вручали медленно поднимается под потолок пепел 
на пол x 2 x начинает 
эксклюзивную экскурсию Очередное подмосковное батонное утро 
и че то еще курю там 
была правда я и не скрываю 
Стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
стало так стало так стало так 
с

In [13]:
rap_gad = tg.generate()
l = rap_gad.split(' ')
for i in range(0, len(l)):
    print(l[i], end=' ')
    if i % 6 == 0:
        print()

Таких 
конченым крысам им Загребущие пасти мы 
любим жить детишкам на этом свете 
в твиттере ответе свидетель  — Moscow 
city прошлое и будущее Невозможно мамы 
Чоли за всем наблюдает Он внимательный 
Поиск огня в холодной темноте Москвы 
реки подъезде я сижу у стены 
все едут на стрелы пьяный подкован 
GFR 1 в стороне от всей 
этой клоунады Орет ребенок следы красного 
на зеленом наш посыл здравым что 
то это значит  — это что 
то значит всё серьёзно писать « 
2 pac » но диги почему 
то не рад сам про стал 
тречков шквал сотни сжатых пружин господа 
не всё потеряно пока отдыхаю пока 
работает народ салют вам на этой 
планете закончатся запасы ганджубаса хотел научиться 
летать … такие песни банальные … 
я знаю что вы в восторге 
от этого ребенка преграждают путь далек 
а это значит  — это что 
то значит всё серьёзно давай поспорим 
исправить карму открыть чакры почистить ауру 
Стрелки терки из за какой нибудь 
точки темнеет в глазах твоих поклонников 
минимум каждый третий уголовник в

In [14]:
url = "http://hip-hop.name/text/krovostok/"
response = r.urlopen(url)
html_doc = response.read()
soup = bs4.BeautifulSoup(html_doc, 'html.parser')
links = []
for link in soup.find_all('a'):
    links.append(link.get('href'))
links = [*filter(lambda x: x is not None and x[:4] == '/kro', links)]

In [15]:
texts_urls = ['http://hip-hop.name' + link for link in links]

In [16]:
%%time
learning_text_krovostok = ''
for url in texts_urls:
    try:
        response = r.urlopen(url)
        html_doc = response.read()
        soup = bs4.BeautifulSoup(html_doc, 'html.parser')
        s = soup.find_all('div', 'entry')[0]
        learning_text_krovostok += s.get_text()
    except:
        pass

CPU times: user 2.67 s, sys: 84.9 ms, total: 2.76 s
Wall time: 24 s


In [17]:
learning_text_krovostok

'\nБакланы жрут коробками план, ву танг клан, хлам\nНy его в спам, прошлогодний снег заебал он всех\nНо речь не о том, дорасскажу потом\nСтою за гаражами в аляске, прижали к стене\nНо греет на кармане полоска острой стали\nКороче приближаться стали ко мне\nНу я, слышь ты, \nРозовый бройлер на анаболиках\nТакие как мы наркоманы бля и алкоголики\nДают оборотку таким как ты\nХуём тебе по подбородку красавица\nСкалится мудачок \nБросается типа убрать пытается\nНо удар убойный \nКак бешек бокс в два косых забитый\nИ переносица в мозг переносится, \nХрипит он и чавкает\nНе нюхнуть колумбийский кокс\nБодяженный сахарной пудрой\nИ ещё заебатая приблуда \nВторому который поменьше\nИсправлен прикус \nИзбавлен малыш от золотых фикс\nПокушает мягкого муда \nИ вот по глине козлик катается\nТот кто среди них был за Тайсона\nКак пиздатый микс, свелась история \nСтало попросторнее\nКость в кость так повелось\nНу и последний Васюня руку к арматуре\nА я ну ты пиздец в натуре дурень \nТыквой чётче думай\

In [18]:
tg = TextGenerator(3, 200)

In [19]:
tg.fit(learning_text_krovostok)

In [20]:
len(learning_text_krovostok)

104197

In [21]:
rap_gad = tg.generate()
l = rap_gad.split(' ')
for i in range(0, len(l)):
    print(l[i], end=' ')
    if i % 6 == 0:
        print()

Да 
здравствует дерзость бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бест 
реально зе бест реально зе бе

In [22]:
tg = TextGenerator(3, 200)
tg.fit(learning_text + learning_text_krovostok)

In [23]:
rap_gad = tg.generate()
l = rap_gad.split(' ')
for i in range(0, len(l)):
    print(l[i], end=' ')
    if i % 6 == 0:
        print()

Прятать 
от родителей тихо под её дверью 
моего подъезда выходит Тамара Константиновна каналов 
красоты не забываем про искренность взгляда 
сильно злой на цирковом руками за 
вас бразерс ценой забывать это все 
еще в этом сечем мы Уж 
в чем чем а в этом 
разбираться лучше ну давай наяривай гитара 
7 ми утра намутили всего лишь 
один человек с огромным стажем зрители 
наша совместка Езда по встречной без 
кипеша и истерики на стене тебе 
это дерьмо оставил вас на целый 
месяц одних трусах менее палевные Интересуйтесь 
и посуйте инфо дальше интересно кто 
там был за то что я 
все еще в этом и есть 
хип хоп не значит всё для 
чего люди рождаются  — в ответ 
не ссы каштанка Запад был послан 
сука клянусь я плакал стонет кормой 
подмахивает вылезает на небо рядом никого 
не было старшего брата от брата 
напрасно не всё потеряно пока мое 
сердце будет биться ка в конец 
ох ела в сквер замусолить лелика 
уйти на пенсию а я уж 
следом свой первый закон бизнеса на 
удачу работы  — устал от дол

In [24]:
def get_textes(author):
    url = "http://hip-hop.name/text/" + author + "/"
    response = r.urlopen(url)
    html_doc = response.read()
    soup = bs4.BeautifulSoup(html_doc, 'html.parser')
    links = []
    for link in soup.find_all('a'):
        links.append(link.get('href'))
    links = [*filter(lambda x: x is not None and x[:4] == '/' + author[:3], links)]
    texts_urls = ['http://hip-hop.name' + link for link in links]
    text = ''
    for url in texts_urls:
        try:
            response = r.urlopen(url)
            html_doc = response.read()
            soup = bs4.BeautifulSoup(html_doc, 'html.parser')
            s = soup.find_all('div', 'entry')[0]
            text += s.get_text()
        except:
            pass
    return text

In [25]:
slim = get_textes('slim')

In [26]:
big_russian_boss = get_textes('big-russian-boss')

In [27]:
noggano = get_textes('noggano')

In [28]:
noggano_generator = TextGenerator(3, 200)
noggano_generator.fit(noggano)

In [29]:
slim_generator = TextGenerator(3, 200)
slim_generator.fit(slim)

In [30]:
big_russian_boss_generator = TextGenerator(3, 200)
big_russian_boss_generator.fit(big_russian_boss)

In [31]:
def generate_text(tg):
    rap_gad = tg.generate()
    l = rap_gad.split(' ')
    for i in range(0, len(l)):
        print(l[i], end=' ')
        if i % 6 == 0:
            print()

In [32]:
all_rappers = learning_text + learning_text_krovostok + slim + big_russian_boss + noggano

In [33]:
len(all_rappers)

776084

In [34]:
anacondaz = get_textes('anacondaz')

In [35]:
%%time
all_rappers_gen = TextGenerator(3, 200)
all_rappers_gen.fit(all_rappers)

CPU times: user 24.3 s, sys: 188 ms, total: 24.5 s
Wall time: 24.6 s


In [36]:
syava = get_textes('syava')

In [ ]:
generate_text(all_rappers_gen)

In [41]:
f = open('russian_rap.txt', 'w')
f

^C
